# Short term Time Series forecasting of Bitcoin & Ethereum using ARIMA and Neural Net LSTM

**Joe Ganser**

The objective of this blog post is to experiment with time series techniques as well as methods in signal/noise extraction to make a prediction on the prices of both bitcoin and ethereum in the last week of August 2017, using all the previous pricing data.

In this short term price forecast, I am predicting the next day's price based upon all the prices leading upto it. Then on the next day, I am repeating this process again, once next day's price is received. Model techniques like this could be used for high frequency trading, not long term investing. This is NOT about predicting long term trends.

Because I am only predicting the next day out, and I am extracting the signal from the noise and what the models ACTUALLY predict is the noise. The predicted noise is then plugged back into the signal function, and that turns into the measurable prediction. This concept is explained a little more below.

Lets take a look at the pricing data upto this point so far;

In [2]:
import IPython
full_plots = '<iframe width="900" height="450" frameborder="0" scrolling="no" src="bitcoin&ethereum_fullplots.html"></iframe>'
IPython.display.HTML(full_plots)

**WORKFLOW** 

My process in making these models this is as follows:

    1.Extract the signal from the noise (find stationarity via dickey fuller test)
    2.Model the noise, and predict the noise for the next day (ARIMA and Neural nets)
    3.Plug the predicted noise into the signal, calculating the price value for the next day
    4.Compare the predicted price of the next day to the actual next day's price
    5.Repeat steps 2 and 3 once when obtaining the actual price of the next day  (use a for loop)

#### MODEL CHOICES: ARIMA & NEURAL NET RECURSION

The model choice was done simply for comparitive experimental reasons, only because ARIMA and Neural Net LSTM can predict time series. Other techniques could be done to, but I just wanted to compare these out of curiosity. Their performance metrics were the cumulative root mean square error across all the days predicted.

#### ASSUMPTIONS

The assumption, which is validated, is that the exponential long term trend shall remain over the last few days of August of 2017. The assumption was made through the techniques of signal noise extraction which are explained below. I do NOT assume that the exponential trend will go on forever.

#### MODEL PREDICTIONS

Here are the final results - the actual predictions of my models.

In [3]:
prediction_plots = '<iframe width="900" height="650" frameborder="0" scrolling="no" src="bitcoin_ethereum_prediction_plots.html"></iframe>'
IPython.display.HTML(prediction_plots)

**MODEL PERFORMANCE METRICS**

So looking at these results, what were the model performance metrics? The performance metric I used to compare the results of the techniques was the summed root mean square error across all the days predicted.

In [4]:
import pandas as pd
metrics = pd.DataFrame({'Neural Net':[42.2849,1.67],'ARIMA':[49.6777,9.73758]}).transpose().rename(columns={0:'Bitcoin RMSE',1:'Ethereum RMSE'})
metrics

,Bitcoin RMSE,Ethereum RMSE
ARIMA,49.6777,9.73758
Neural Net,42.2849,1.67000


As we can see, the RMSE of the Neural Nets was lower in this case and was thus the better performing model. Of course that's not to make a general statement that Neural Nets is better than ARIMA, because there are many examples where Neural Net time series analysis may be a better choice.

So how did I solve this problem? As mentioned above in the work flow description, the first step is to extract the signal from the noise.

###### **EXTRACTING THE SIGNAL FROM THE NOISE USING THE AUGMENTED DICKEY FULLER TEST**

![Signal_noise](Signal_noise.png)


The augmented dickey fuller test is a statistical test to determine "stationarity". Stationarity occurs when a series of data is transformed into format which clearly identifies the signal and all we are left with is noise. More specifically, stationarity occurs when we put the time series data is put into it's correct inverse function. With outputs similar to a Z-test, the test tells us if the new format we have has clearly identified the signal within the data. The null hypothesis of the Dickey Fuller test is that the signal and noise are NOT decoupled. The strategy behind using this is that by extracting the signal, we model and predict the noise and plug those predictions back into the signal.

**THE BEST FIT FUNCTION & IT'S INVERSE**

Knowing the best fitting function is not enough to model the time series. What is needed is not only a best fit function, but that function's inverse as well. By putting the data of the time series into the inverse function, we can get the noise values. It's noise values that are put into the Dickey Fuller test.

![INVERSE](InverseFunction.png)




    Extracting the signal from the noise can be summed into three steps
    1. Find the a function similar to the time series data.
    2. Find that function's inverse.
    3. Plug the time series data into the inverse, and plug that transformed data set into the Dickey Fuller Test
    *If we haven't rejected the null hypothesis in step 3 try step 1 over starting with a different function.


By looking at the very first plot, we can see that the trend of the cryptocurrency prices are in some way exponential. Thus, I can hypothesize that the signal is exponential in form.

![bitcoin_vs_exponential](bitprice_vs_exponential_curve.png)


The actual function form that allowed me to seperate the signal from the noise was indeed *exponential in form* but not simply exponential. It was by using this function's inverse that I could pass the dickey fuller test.

The actual function that worked was this:

![Function](function.png)

These function transformations worked with both the bitcoin & ethereum data.

The code that puts the data into stationary form is:

In [ ]:
noise = (np.log(data['Price'])-np.log(data['Price'].shift())).dropna()

Where the '.shift()' cooresponds to the day before.

**FAILING AND THEN PASSING THE DICKEY FULLER TEST**

In the tabs below you'll see the attempt at passing the Dickey Fuller test without perfmoring any transformation, and then (botton of each tab), the Dickey Fuller test is passed (meaning we can reject the null hypothesis, and the data is stationary.)

In [4]:
dickeyfuller = '<iframe width="900" height="700" frameborder="0" scrolling="no" src="bitcoin&ethereum_dickey_fuller.html"></iframe>'
IPython.display.HTML(dickeyfuller)

So now I have clearly identified the stationary form of the time series. These stationary forms can be plugged into the models to predict the noise, and using the noise predictions we can plug those back into the signal.

## MODEL BUILDING & MAKING PREDICTIONS

**So the prediction steps are:**

    1. Plug the noise into the model
    2. Train the model on the history of the noise
    3. Predict the noise of the next day
    4. Plug the next day's predicted noise into the signal function.
    5. Add the actual price of the next day to the price data set, transform it into stationarity and start again at step 1.

**MODELLING WITH ARIMA**

There are several steps to building an ARIMA model. ARIMA operates on three parameters. Based upon the plots of auto-correlation, partial auto-correlation and the amount of Bayesian information criterion we select these three parameters. Ofcourse, the end goal is to minimize the root mean squared error of the model.

**AUTO CORRELATION & PARTIAL AUTOCORRELATION PLOTS **

The plots of auto-correlation & partial auto-correlation for bitcoin & ethereums stationary formats were:
![correlation](correlation_plots.png)

Using these plots I can estimate the ARIMA parameters (p,q) of the parameters (p,q,d) for ARIMA models. Just by looking at the graphs it can be estimate that p & q are around no more than 7 each. After grid searching through lots of combinations it was found that **Bitcoins ARIMA parameters were (p=0,d=1,q=1)** and **Ethereum's ARIMA parameters were (p=2,d=0,q=0)**

In [5]:
import pandas as pd
import numpy as np
from datetime import date
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
bitcoin =pd.read_csv('bitcoin.csv').drop('Unnamed: 0',axis=1)
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])
bitcoin.set_index('Date',inplace=True)
bits_log_shift = (np.log(bitcoin['Price']) - np.log(bitcoin['Price']).shift()).dropna()
ethereum = pd.read_csv('ethereum.csv').drop('Unnamed: 0',axis=1)
ethereum['Date'] = pd.to_datetime(ethereum['Date'])
ethereum.set_index('Date',inplace=True)
eth_log_shift = (np.log(ethereum['Price']) - np.log(ethereum['Price']).shift()).dropna()

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


**Here is the ARIMA model code:**

In [6]:
def ARIMA_predictions(original_series,stationary_series,parameters, days_out):
    # Here we predict the noise
    train = stationary_series[:-days_out]
    # test the model on the last N points of the data
    test = stationary_series[-days_out:]
    #
    history = [x for x in train]

    train.dropna(inplace=True)
    test.dropna(inplace=True)
    predicted_values = []
    tested = []
    
    #1. Plug the noise into the model
    #2. Train the model on the history of the noise
    #3. Predict the noise of the next day
    #4. Plug the next day's predicted noise into the signal function.
    #5. Add the actual price of the next day to the price data set, transform it into stationarity and start again at step 1.
    
    for i in range(len(test)):
        model = ARIMA(history, order=parameters)
        model_fit = model.fit(disp=0)
        yhat = float(model_fit.forecast(steps=1)[0])
        predicted_values.append(yhat)
        tested_values = list(test)[i]
        tested.append(tested_values)
        history.append(tested_values)
    predictions_series = pd.Series(predicted_values, index = test.index)
    
    # This part couples the signal to the noise.
    a = original_series.loc[original_series.index[-(days_out+1):]]['Price']
    b = np.exp(predictions_series)
    full_predictions = pd.DataFrame(a*b,columns=['Predicted with ARIMA']).dropna()
    df = pd.concat([original_series.loc[original_series.index[-days_out:]],full_predictions],axis=1)
    error = str(np.sqrt(mean_squared_error(df['Price'],df['Predicted with ARIMA'])))
    print("ARIMA Root Mean Squared Error: ",error)
    df.index.name = None
    df[['Price','Predicted with ARIMA']] = df[['Price','Predicted with ARIMA']].apply(lambda x: round(x,2))
    return df


print("ARIMA bitcoin predictions")
bitcoin_ARIMA = ARIMA_predictions(bitcoin,bits_log_shift,(0,1,1),7)
bitcoin_ARIMA

ARIMA bitcoin predictions
ARIMA Root Mean Squared Error:  3.898537928223503


,Price,Predicted with ARIMA
2017-08-23,4318.35,4321.92
2017-08-24,4364.41,4368.25
2017-08-25,4352.30,4356.23
2017-08-26,4345.75,4349.69
2017-08-27,4390.31,4394.20
2017-08-28,4597.31,4601.04
2017-08-29,4583.02,4587.37


The neural net system was based upon sequential, dense LSTM modelling techniques. Like the ARIMA model, it was dependent upon a stationary time series input, modeling the noise and then putting the noise predictions back into the signal. 

In [7]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM



def Neural_Net_predictions(original_time_series, stationary_time_series, days_out,nb_epoch,neurons):
    # note all these "sub" functions are used on the stationary time series.
    # The neural nets are used to predict the noise. Once the noise is predicted
    # Its plugged back into the signal
    
    X = stationary_time_series
    
    # Step 2
    # Break the time series into shifted components. Each column is a shifted value 
    # previously in the time series
    def timeseries_to_supervised(data, lag=1):
        df = pd.DataFrame(data)
        columns = [df.shift(i) for i in range(1, lag+1)]
        columns.append(df)
        df = pd.concat(columns, axis=1)
        df.fillna(0, inplace=True)
        return df
    
    # Step 3
    # We must put the time series onto the scale acceptable by the activation functions
    def scale(train, test):
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler = scaler.fit(train)
        train = train.reshape(train.shape[0],train.shape[1])
        train_scaled = scaler.transform(train)
        test = test.reshape(test.shape[0],test.shape[1])
        test_scaled = scaler.transform(test)
        return scaler, train_scaled, test_scaled
    
    
    # Step 4
    def fit_lstm(train, batch_size, nb_epoch, neurons):
        X, y = train[:, 0:-1], train[:, -1]
        X = X.reshape(X.shape[0], 1, X.shape[1])
        model = Sequential()
        model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        for i in range(nb_epoch):
            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
            model.reset_states()
        return model

    
    # Step 5
    def forecast_lstm(model, batch_size, X):
        X = X.reshape(1, 1, len(X))
        yhat = model.predict(X, batch_size=batch_size)
        return yhat[0,0]
    
    #Step 6
    def invert_scale(scaler, X, value):
        new_row = [x for x in X] + [value]
        array = np.array(new_row)
        array = array.reshape(1, len(array))
        inverted = scaler.inverse_transform(array)
        return inverted[0, -1]

    # Now use all the above functions
    supervised = timeseries_to_supervised(X,1)
    supervised_values = supervised.values
    train, test = supervised_values[0:-days_out], supervised_values[-days_out:]
    scaler, train_scaled, test_scaled = scale(train, test)
    train_reshaped = train_scaled[:,0].reshape(len(train_scaled),1,1)
    lstm_model = fit_lstm(train_scaled,1,nb_epoch,neurons)
    
    #Step 7
    predictions = []
    for i in range(len(test_scaled)):
        #Make one step forecast
        X, y = test_scaled[i,0:-1], test_scaled[i,-1]
        yhat = forecast_lstm(lstm_model,1,X)
        #invert scaling
        yhat = invert_scale(scaler, X, yhat)
        #store forecast
        predictions.append(yhat)
    
    # Step 8
    # This part plugs it back into the signal
    predictions_series = pd.Series(predictions, index = original_time_series.index[-days_out:])
    a = original_time_series.loc[original_time_series.index[-(days_out+1):]]['Price']
    b = np.exp(predictions_series)
    full_predictions = pd.DataFrame(a*b,columns=['Predicted with Neural Nets']).dropna()
    df = pd.concat([original_time_series.loc[original_time_series.index[-days_out:]],full_predictions],axis=1)
    error = str(np.sqrt(mean_squared_error(df['Price'],df['Predicted with Neural Nets'])))
    print("Neural Net Root Mean Squared Error: ",error)
    df.index.name=None
    df[['Price','Predicted with Neural Nets']] = df[['Price','Predicted with Neural Nets']].apply(lambda x: round(x,2))
    return df

print('Bitcoin Neural Net Predictions')
bitcoin_NN = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=55,neurons=175)
bitcoin_NN

Using TensorFlow backend.


Bitcoin Neural Net Predictions
Neural Net Root Mean Squared Error:  49.40448539455479


,Price,Predicted with Neural Nets
2017-08-23,4318.35,4369.75
2017-08-24,4364.41,4423.16
2017-08-25,4352.30,4397.79
2017-08-26,4345.75,4384.69
2017-08-27,4390.31,4430.15
2017-08-28,4597.31,4643.98
2017-08-29,4583.02,4643.36
